In [2]:
%pylab inline --no-import-all
import numpy as np
import scipy as sp
import pylab as pl
import matplotlib.pyplot as plt
import statistics as st
import pandas as pd
from scipy.stats import norm
import time
from sklearn.ensemble import RandomForestRegressor


Populating the interactive namespace from numpy and matplotlib


In [3]:
"Function creating N paths for the Black-Scholes formula"

def stocks_paths(S_0,r,sigma,dt,N,m,mu):  
    S = np.zeros((m + 1, N))
    dB = np.zeros((m + 1, N))
    S[0] = S_0
    for t in range(1,m + 1):
        X=np.random.standard_normal(size=N)
        S[t]=S[t-1]*np.exp((mu-sigma*sigma/2)*dt+sigma*np.sqrt(dt)*X)
        dB[t] =  np.sqrt(dt) *X
    S_dB={'Stocks' : S,'delta_B': dB}
    return S_dB    

In [5]:
def pricing_option(mu,R,K,r,sigma,N,m,S_0,T,RF_n_trees, RF_max_leaf_nodes):
    '''
    Function to generte stock paths.
    
    Parameters
    ==========
    S_0             : float
                      positive, initial Stock Value
    mu              : float
                      drift
    K               : float 
                      Strike price
    T               : float 
                      Maturity time 
    sigma           : float 
                      volatility 
    r               : float 
                      lending interest rate 
    R               : float 
                      borrowing interest rate 
    N               : int 
                      Number of paths generated      
    m               : int 
                      number of steps
    d               : int 
                      polynomial fit degree
    
    Returns
    =======
    V_opt : float
            Price of the European option
    '''  
    
    dt=T/m                             #Time-step
    df=1/(1+r*dt)                      #Discount factor
    theta=(r - mu)/sigma
    
    S_dB=stocks_paths(S_0,r,sigma,dt,N,m,mu)       # Dictionnary with both the delta_B and Stocks generated

    S=S_dB['Stocks']
    dB=S_dB['delta_B']

    Y=np.maximum(S[-1]-K,0)                 #price of the option at time T = Initialization for a call
   
    for t in range (m-1, 0, -1):            #Iteration over time backwardly 
        X=S[t]
        X = X[:, None]
        "Regression for Z"
        rf=RandomForestRegressor(n_estimators=RF_n_trees,
                                     max_leaf_nodes=RF_max_leaf_nodes,
                                     oob_score=False, 
                                     n_jobs=-1)
        
        rf.fit(X,Y*dB[t])
        Z = rf.predict(X) *  (1./ dt)
        
        "regression for Y"
        rf.fit(X, Y)
        J = rf.predict(X)
        #Y = np.polyval(reg,X)-Y*r*dt-theta*Z*dt+np.minimum(Y-(1/sigma)*Z,0)*(R-r)*dt
        Y=df*(J-theta*Z*dt+np.minimum(Y-(1./sigma)*Z, 0)*(R-r)*dt)
  
    Y_opt= df*sum(Y)/N  
    return (Y_opt)

In [6]:
"""Pricing with BS formula"""

def black_scholes(K,r,sigma, S_0,T): 
    d1=1/(sigma*np.sqrt(T))*(np.log(S_0/K)+(r+0.5*sigma*sigma)*T)
    d2=d1-sigma*np.sqrt(T)
    dscnt=np.exp(-r*T)
    Call=-norm.cdf(d2)*K*dscnt+norm.cdf(d1)*S_0
    return (Call)

In [11]:
                                """Pricing by Random Forest Regressor"""

start = time.time()

  
    
"Parameters"
T=0.5
m=12
K=100
S_0=100
r=0.04
sigma=0.2
N=10000
mu=0.06
R=0.06
RF_n_trees = 10
RF_max_leaf_nodes= 5

"We price"    

price_BSDE=pricing_option(mu,R,K,r,sigma,N,m,S_0,T, RF_n_trees,RF_max_leaf_nodes)
price_BScholes=black_scholes(K,R,sigma, S_0,T)
print ("price ", price_BSDE) 
print ("price BScholes ", price_BScholes)

end = time.time()
print(end - start,"s") 

price  7.23060994686
price BScholes  7.15589605611
6.992681503295898 s


In [ ]:
                                """Simulations : m varying"""

    

"Parameters"
T=0.25
m=12
K1=95
K2=105
S_0=100
r=0.01
sigma=0.2
mu=0.05
R=0.06
RF_n_estimators = 100
d=5


N_range=[100,1000,10000]
Prices=[]

for N in N_range:
    Prices_int=np.zeros(15)
    for i in range (15): 
        Prices_int[i]=pricing_option(mu,R,K1,K2,r,sigma,N,m,S_0,T,d, RF_n_estimators)
        
    Prices.append(Prices_int)
  

    
df = pd.DataFrame(np.transpose(Prices), columns=['100','1000','10000'])

df.plot(kind='box')
plt.show()

"We price"    

